# Psychometric Analysis

## To be used with Google Colab
1. https://colab.research.google.com/
2. Choose **Github**
3. Paste the url **`https://github.com/shlomihod/psypsy/blob/master/psychometric-analysis.ipynb`**
4. Click on **NEW PYTHON 3 NOTEBOOK**
5. Click on **Runtime** and then **Run All**
6. Upload **`responses.xlsx`** file
7. Print as PDF


## Useful Reference:
https://www.creighton.edu/sites/www12.creighton.edu/files/PtT-Exam%20Analysis.pdf

## Setup

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pylab as plt
import seaborn as sns

from IPython.display import display

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 5.0)

## Upload Data

In [ ]:
DATA_FILENAME = 'responses.xlsx'

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
          name=fn, length=len(uploaded[fn])))

In [ ]:
assert DATA_FILENAME in uploaded

with open(DATA_FILENAME, 'wb') as f:
    f.write(uploaded[DATA_FILENAME])

## Load Data

In [ ]:
responses = pd.read_excel(DATA_FILENAME, sheet_name='responses')

answer_key_df = pd.read_excel(DATA_FILENAME, sheet_name='key')

ITEM_COLS = {col : col + '_is_correct'
             for col in responses.columns if 'item' in col}

## Validate Data

In [ ]:
assert not responses.isnull().any().any()

assert not responses.duplicated().any()
assert not responses['id'].duplicated().any()
assert not responses['name'].duplicated().any()

assert responses['gender'].isin({'M', 'F'}).all()
assert responses['class'].isin({10, 11, 12}).all()
assert responses[list(ITEM_COLS.keys())].isin({1, 2, 3, 4}).all().all()

In [ ]:
assert all(answer_key_df.columns == list(ITEM_COLS.keys()))
assert len(answer_key_df) == 1

answer_key = answer_key_df.iloc[0].to_dict()

## Calculate Score and `is_correct` Columns

In [ ]:
for col, col_is_correct in ITEM_COLS.items():
    responses[col_is_correct] = (responses[col]
                                      == answer_key[col])

In [ ]:
responses['score'] = responses[list(ITEM_COLS.values())].sum(axis=1)

## Score - Descriptive Statistics

In [ ]:
(responses['score']
 .describe(percentiles=np.arange(0, 1, 0.1))
 .round(3))

In [ ]:
sns.distplot(responses['score'])

plt.title('Score Distribution');

In [ ]:
sns.distplot(responses['score'],
             hist_kws={'cumulative': True},
             kde_kws={'cumulative': True})

plt.title('Score Cumulative Distribution');

## Reliability - Internal Consistency 

### Code & Tests

In [ ]:
def cronbach_alpha_KR20(items):
    """Calculate KR20 for dichotomous items.
    
    # https://github.com/anthropedia/tci-stats
    """
    items = pd.DataFrame(items)
    items_count = items.shape[1]
    
    # note: probably ddof need to be changed to 1 for non-dichotomous items.
    variance_sum = items.var(axis=0, ddof=0).sum()
    
    total_var = items.sum(axis=1).var(ddof=1)
    
    return (items_count / (items_count - 1) *
            (1 - variance_sum / total_var))

In [ ]:
# http://www.hr-survey.com/WpAssessmentHandbook.htm
# Estimating Reliability using the Kuder-Richardson Formula 20


ca_test_data ="""N	1	2	3	4	5	6	7	8	9	10	11	12
A	1	1	1	1	1	1	1	0	1	1	1	1
B	1	1	1	1	1	1	1	1	0	1	1	0
C	1	1	1	1	1	1	1	1	1	0	0	0
D	1	1	1	0	1	1	0	1	1	0	0	0
E	1	1	1	1	1	0	0	1	1	0	0	0
F	1	1	1	0	0	1	1	0	0	1	0	0
G	1	1	1	1	0	0	1	0	0	0	0	0
H	1	1	0	1	0	0	0	1	0	0	0	0
I	1	1	1	0	1	0	0	0	0	0	0	0
J	0	0	0	1	1	0	0	0	0	0	0	0
"""

In [ ]:
ca_test_df = (pd.DataFrame({col: data
                  for col, *data in
                  zip(*(row.split()
                        for row in ca_test_data.splitlines()))
                 })
              .set_index('N')
              .astype(int))

assert np.isclose(cronbach_alpha_KR20(ca_test_df), 0.8, atol=0.001)

### Calculating Reliability

| Cronbach's alpha | Internal consistency |
|------------------|----------------------|
| 0.9 ≤ α          | Excellent            |
| 0.8 ≤ α < 0.9    | Good                 |
| 0.7 ≤ α < 0.8    | Acceptable           |
| 0.6 ≤ α < 0.7    | Questionable         |
| 0.5 ≤ α < 0.6    | Poor                 |
| α < 0.5          | Unacceptable         |

In [ ]:
cronbach_alpha_KR20(responses[list(ITEM_COLS.values())])

## Item Analysis

In [ ]:
def pbs(item):
     return stats.pointbiserialr(item,
                                 responses['score'])[0]

item_statistics = (responses[list(ITEM_COLS.values())]
                   .agg(['mean', 'std', pbs])
                   .transpose()
                   .rename({'mean': 'proportion'}, axis=1)
                  )

item_statistics['reliability_without'] = item_statistics.index.map(lambda item: 
                                        cronbach_alpha_KR20(responses[list(ITEM_COLS.values())]
                                                            .drop(item, axis=1)))

item_statistics = (item_statistics.round(3)
                                  .rename(index=lambda x: x.split('_')[0]))

item_statistics

In [ ]:
item_statistics[['proportion', 'pbs']].plot()

plt.title('Difficulty and Discrimination per Item');

In [ ]:
item_statistics['proportion'].plot(kind='bar')

plt.title('Difficulty per Item');

In [ ]:
item_statistics.sort_values('proportion')

### Per Item Analysis
(with colab anti-collapse ugly hack)

In [ ]:
def perform_responses_per_item_analysis(df, cols):
    for col in cols:

        responses_per_item = (df
                              .groupby([col])['score']
                              .agg(['size', 'mean']))

        responses_per_item['proportion'] = (responses_per_item['size']
                                            / responses_per_item['size']).sum()

        responses_per_item = responses_per_item.round(3)

        responses_per_item = responses_per_item[['mean', 'size', 'proportion']]

        responses_per_item.index = responses_per_item.index.astype(str)

        responses_per_item = responses_per_item.rename(index = {str(answer_key[col]):
                                                                '*' + str(answer_key[col])})
        display(responses_per_item)

In [ ]:
perform_responses_per_item_analysis(responses, list(ITEM_COLS.keys())[:5])

In [ ]:
perform_responses_per_item_analysis(responses, list(ITEM_COLS.keys())[5:10])

In [ ]:
perform_responses_per_item_analysis(responses, list(ITEM_COLS.keys())[10:15])

In [ ]:
perform_responses_per_item_analysis(responses, list(ITEM_COLS.keys())[15:20])

In [ ]:
perform_responses_per_item_analysis(responses, list(ITEM_COLS.keys())[20:25])

In [ ]:
perform_responses_per_item_analysis(responses, list(ITEM_COLS.keys())[25:30])

## Analysis By Gender

In [ ]:
responses_by_gender = responses.groupby('gender')

In [ ]:
(responses_by_gender['score']
 .describe(percentiles=np.arange(0, 1, 0.1))
 .round(3).T)

In [ ]:
for label, group in responses_by_gender:
    sns.distplot(group['score'], label=label)

plt.legend()
plt.title('Score Distribution by Gender');

In [ ]:
for label, group in responses_by_gender:
    sns.distplot(group['score'],
                 hist_kws={'cumulative': True},
                 kde_kws={'cumulative': True})

plt.legend()
plt.title('Score Cumulative Distribution by Gender');

## Analysis By Class

In [ ]:
responses_by_class = responses.groupby('class')

In [ ]:
(responses_by_class['score']
 .describe(percentiles=np.arange(0, 1, 0.1))
 .round(3).T)

In [ ]:
for label, group in responses_by_class:
    sns.distplot(group['score'], label=label)

plt.legend()
plt.title('Score Distribution by Class');

In [ ]:
for label, group in responses_by_class:
    sns.distplot(group['score'],
                 hist_kws={'cumulative': True},
                 kde_kws={'cumulative': True})

plt.legend()
plt.title('Score Cumulative Distribution by Class');